Steps:
- Preprocessing --> removing space, punctuations, stopwords, lemmatization
- Create Dictionary
- Create bag of words corpus
- Apply tf-idf on bow corpus
- Create a tf-idf vector of docs 
- Create a vector of embeddings using spacy 'en' model
- Get a dot product of tf-idf vector and embeddings vector

Repeat above steps for a new document and
calculate cosine similarity between the two embeddings

In [0]:
import os
import re
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import pandas as pd
import numpy as np
nlp  = spacy.load('en_core_web_sm')
from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.matutils import sparse2full
import scipy

In [0]:
file = '/home/inno/innovation/Shyam/Entity_Extraction/SOWEntityExtraction/output/sow_entities.csv'
df = pd.read_csv(file, names = ['FirstParty','SecondParty', 'EffectiveDate','EndDate','Duration','Contract','Services','PaymentTerms'])
print(df.shape)
df.head()

FileNotFoundError: ignored

In [0]:
def keep_token(t):
    return (t.is_alpha and 
            not (t.is_space or t.is_punct or 
                 t.is_stop or t.like_num))

def lemmatize_doc(doc):
    return [ t.lemma_ for t in doc if keep_token(t)]

In [0]:
def get_embeddings(docs):
    docs = [lemmatize_doc(nlp(doc)) for doc in docs]
    docs_dict = Dictionary(docs)
    #test_docs_dict.filter_extremes(no_below=20, no_above=0.2)
    docs_dict.compactify()
    docs_corpus = [docs_dict.doc2bow(doc) for doc in docs]
    model_tfidf = TfidfModel(docs_corpus, id2word=docs_dict)
    docs_tfidf  = model_tfidf[docs_corpus]
    docs_vecs   = np.vstack([sparse2full(c, len(docs_dict)) for c in docs_tfidf])
    tfidf_emb_vecs = np.vstack([nlp(docs_dict[i]).vector for i in range(len(docs_dict))])
    docs_emb = np.dot(docs_vecs, tfidf_emb_vecs)
    return docs_emb

In [0]:
docs = []
for i in range(len(df.Services)):
    if(type(df.Services[i]) != float):
        docs.append(df.Services[i])
docs_emb = get_embeddings(docs)

In [0]:
def calculate_cosine_distance(test_docs_emb, docs_emb):
    max_cosine = 0    
    para_with_max_cosine = None
    for i in range(test_docs_emb.shape[0]):
        vector_1 = test_docs_emb[i]    
        for j in range(docs_emb.shape[0]):
            vector_2 = docs_emb[j]        
            cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
            cosine = round((1-cosine)*100,2)        
            if(not(np.isnan(cosine))):
                if(cosine > max_cosine):
                    max_cosine = cosine
                    para = i
                            
    return max_cosine, para

In [0]:
path = '/home/inno/innovation/Shyam/Entity_Extraction/data/sow_converted_text/'
files = os.listdir(path)
for i, file in enumerate(files):
    if(i<10):
        f = open(path + file, "r")
        text = f.read()
        test_docs = text.split('.\n\n')
        test_docs_emb = get_embeddings(test_docs)    
        max_cosine, para = calculate_cosine_distance(test_docs_emb, docs_emb)
        print('Below paragraphs #{} in file {} is with max cosine similarity of {} \n {} \n'.format(para, file, max_cosine, test_docs[para].replace("\n", " ")))                

/home/inno/.local/lib/python3.6/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Below paragraphs #20 in file D29087.pdf.out.html.txt is with max cosine similarity of 99.2 
  Virtustream agrees to keep commercially reasonable records of all expenses to support claims for   reimbursement from Customer. Expenses for time spent by Virtustream in traveling to and from   Customer facilities are not reimbursable. Virtustream shall utilize reasonable efforts to minimize the   cost of expenses incurred as follows: no first class travel, no luxury car rentals, no luxury hotel rentals,   no Juxury restaurants, no utilization of airline with preferable frequent flier program when   significantly less expensive alternative is available. In addition, Virtustream will work with Wolters   Kluwer Financia! Services travel services personnel to obtain discounts or other benefits available 

Below paragraphs #1 in file D15044.pdf.out.html.txt is with max cosine similarity of 99.13 
  1. Services: DGSL shal! provide the following services to Loislaw:   Data capture and XML conversion

References:

http://dsgeek.com/2018/02/19/tfidf_vectors.html

https://stackoverflow.com/questions/8897593/how-to-compute-the-similarity-between-two-text-documents

https://stackoverflow.com/questions/30285706/detecting-similar-paragraphs-in-two-documents
